In [ ]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist
import time

from processor.dielectron_mass import dielectron_mass
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive

year = 2017

In [ ]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples
from Tools.config_helpers import redirector_ucsd, redirector_ucsd_mini, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

from processor.meta_processor import get_sample_meta

overwrite = True
local = True

# load the config and the cache
cfg = loadConfig()

cacheName = 'dielectron_mass_data'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
fileset = make_fileset(['Data'], year, redirector=redirector_ucsd, small=False)

# make a fileset, taking the definitions in Tools/nano_mapping.py
mc_fileset = make_fileset(['DY', 'TT/TW', 'WZ/ZZ'], year, redirector=redirector_ucsd, small=False)
meta = get_sample_meta(mc_fileset, samples)

# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress
    
    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)
    
    def unique(filename):
        file, ext = os.path.splitext(filename)
        counter = 0
        while os.path.exists(filename):
            counter += 1
            filename = file + str(counter) + ext
        return filename

    tstart = time.time()
    
    from dask.distributed import performance_report
    fname = unique("dask/dask-report_chunksize=" + str(chunksize/1000) + "K.html")
    
    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
        'savemetrics': True
    }
    exe = processor.dask_executor


if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')
    
else:
    print ("I'm running now")
    output = processor.run_uproot_job(
        fileset,
        "Events",
        dielectron_mass(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
        )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()
    

In [ ]:
# import the plotting libararies: matplotlib and mplhep

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np


# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import makePlot, scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
pt_fine_bins   = hist.Bin('pt', r'$p_{T}\ (GeV)$', 100, 0, 500)
pt_rebin       = hist.Bin('pt', r'$p_{T}\ (GeV)$', 50, 0, 100)
met_pt         = hist.Bin('pt', r'$p_{T}\ (GeV)$', 60, 0, 300)
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
eta_rebin      = hist.Bin('eta', r'$\eta $', 25, -2.5, 2.5)    
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)
mass_bins      = hist.Bin('mass', r'$mass (GeV/c^2)$', 15, 76, 106)

lumi = {2016: 36, 2017: 42, 2018: 60}
nano_mappings = nano_mapping(year)

# 1D Histograms

In [ ]:
from yahist import Hist1D, Hist2D

In [ ]:
def get_total(histos, keys):
        tmp = Hist1D.from_bincounts(np.zeros(len(histos[keys[0]].counts)), histos[keys[0]].edges, )
        for key in keys:
            tmp += histos[key]
        return tmp

def add_uncertainty(hist, ax, ratio=False):
    opts = {'step': 'post', 'label': 'Uncertainty', 'hatch': '///',
                    'facecolor': 'none', 'edgecolor': (0, 0, 0, .5), 'linewidth': 0, 'zorder':10.}
    
    if ratio:
        down = np.ones(len(hist.counts)) - hist.errors/hist.counts
        up = np.ones(len(hist.counts)) + hist.errors/hist.counts
    else:
        down = hist.counts-hist.errors
        up = hist.counts+hist.errors
    ax.fill_between(x=hist.edges, y1=np.r_[down, down[-1]], y2=np.r_[up, up[-1]], **opts)

In [ ]:
import gzip
import pickle
import cloudpickle

In [ ]:
outname = 'dielectron_data_ss_'+str(year)
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')

In [ ]:
path = 'histos/dielectron_mc_NLO_ss_2018.pkl.gz'
with gzip.open(path) as fin:
    mc_output= pickle.load(fin)
    
path = 'histos/dielectron_data_ss_2018.pkl.gz'
with gzip.open(path) as fin:
    output= pickle.load(fin)

In [ ]:
tmp1 = output['dilep_mass1'].copy()
tmp1 = tmp1.rebin('mass', mass_bins)
tmp1 = tmp1.rebin('pt', pt_fine_bins)

tmp1_ss = output['dilep_mass4'].copy()
tmp1_ss = tmp1_ss.rebin('mass', mass_bins)
tmp1_ss = tmp1_ss.rebin('pt', pt_fine_bins)

h1 = Hist1D.from_bincounts(
    tmp1.sum('pt', 'dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('pt', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h2 = Hist1D.from_bincounts(
    tmp1.sum('mass', 'dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('mass', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)


h1_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('pt', 'dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('pt', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h2_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('mass', 'dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('mass', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

tmp2 = scale_and_merge(mc_output['dilep_mass1'], meta, mc_fileset, nano_mappings, lumi=lumi[year])
tmp2 = tmp2.rebin('mass', mass_bins)
tmp2 = tmp2.rebin('pt', pt_fine_bins)

h1_DY = Hist1D.from_bincounts(
    tmp2.sum('pt', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp2.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h1_TT_TW = Hist1D.from_bincounts(
    tmp2.sum('pt', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp2.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

h1_VV = Hist1D.from_bincounts(
    tmp2.sum('pt', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp2.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

h2_DY = Hist1D.from_bincounts(
    tmp2.sum('mass', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('mass', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h2_TT_TW = Hist1D.from_bincounts(
    tmp2.sum('mass', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('mass', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

h2_VV = Hist1D.from_bincounts(
    tmp2.sum('mass', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('mass', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

scale_factor = (np.sum(h1_ss.counts)-np.sum(h1_TT_TW.counts)-np.sum(h1_VV.counts))/np.sum(h1.counts)
print(scale_factor)
h1 = h1 * scale_factor
h2 = h2 * scale_factor

h1_DY = h1_DY * scale_factor
h2_DY = h2_DY * scale_factor

In [ ]:
hists = {'mass_data_OS':  h1,
         'mass_data_SS':  h1_ss,
         'mass_DY_OS':    h1_DY,
         'mass_TT_TW_OS': h1_TT_TW,
         'mass_VV_OS':    h1_VV,
}

hists['mass_TT_TW_OS'].label = '$TT/TW\ (SS)$'
hists['mass_TT_TW_OS'].color = '#8AC926'

hists['mass_DY_OS'].label = '$DY\ (SS)$'
hists['mass_DY_OS'].color = '#FFCA3A'

hists['mass_VV_OS'].label = '$WZ/ZZ\ (SS)$'
hists['mass_VV_OS'].color = '#C93126'

keys = ['mass_TT_TW_OS', 'mass_VV_OS', 'mass_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['mass_data_OS']).divide(hists['mass_TT_TW_OS']+hists['mass_DY_OS']+hists['mass_VV_OS'])
ratio = (hists['mass_data_SS']).divide(hists['mass_data_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['mass_DY_OS'].edges,
    w2=[(hists[x].errors)**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, np.sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['mass_data_OS'].counts],
    hists['mass_data_OS'].edges,
    w2=(hists['mass_data_OS'].errors)**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['mass_data_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['mass_data_SS'].counts],
    hists['mass_data_SS'].edges,
    w2=(hists['mass_data_SS'].errors)**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['mass_data_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    ratio.counts,
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['mass_data_OS'].counts/((hists['mass_TT_TW_OS'].counts+hists['mass_DY_OS'].counts+hists['mass_VV_OS'].counts)**2)],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$mass_{ee}\ (GeV)$')
rax.set_ylabel(r'Obs/Pred')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
rax.axhline(y=1, color='r', linestyle='-')
#ax.set_ylim(0.1,1e5)

#add_uncertainty(total_mc, rax, ratio=True)


ax.legend(prop={"size":11})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data_ss_'+str(year)+'.png')

In [ ]:
hists = {'pt_data_OS':   h2,
         'pt_data_SS':   h2_ss,
         'pt_DY_OS':     h2_DY,
         'pt_TT_TW_OS':  h2_TT_TW,
         'pt_VV_OS':     h2_VV,
        }

hists['pt_TT_TW_OS'].label = '$TT/TW\ (SS)$'
hists['pt_TT_TW_OS'].color = '#8AC926'

hists['pt_DY_OS'].label = '$DY\ (SS)$'
hists['pt_DY_OS'].color = '#FFCA3A'

hists['pt_VV_OS'].label = '$WZ/ZZ\ (SS)$'
hists['pt_VV_OS'].color = '#C93126'

keys = ['pt_TT_TW_OS', 'pt_VV_OS', 'pt_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_data_OS']).divide(hists['pt_TT_TW_OS']+hists['pt_DY_OS']+hists['pt_VV_OS'])
ratio = (hists['pt_data_SS']).divide(hists['pt_data_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['pt_DY_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, np.sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['pt_data_OS'].counts],
    hists['pt_data_OS'].edges,
    w2=hists['pt_data_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (SS pred)', np.sum(hists['pt_data_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['pt_data_SS'].counts],
    hists['pt_data_SS'].edges,
    w2=hists['pt_data_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (SS obs)', np.sum(hists['pt_data_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['pt_data_OS'].counts/(hists['pt_TT_TW_OS'].counts+hists['pt_DY_OS'].counts+hists['pt_VV_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$p_{T_{ee}}\ (GeV)$')
rax.set_ylabel(r'Obs/Pred')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
rax.axhline(y=1, color='r', linestyle='-')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zpt_data_ss_'+str(year)+'.png')

In [ ]:
tmp1 = output['electron_data1'].copy()
tmp1 = tmp1.rebin('pt', pt_rebin)
tmp1 = tmp1.rebin('eta', eta_rebin)

tmp2 = output['electron_data2'].copy()
tmp2 = tmp2.rebin('pt', pt_rebin)
tmp2 = tmp2.rebin('eta', eta_rebin)

tmp1_ss = output['electron_data7'].copy()
tmp1_ss = tmp1_ss.rebin('pt', pt_rebin)
tmp1_ss = tmp1_ss.rebin('eta', eta_rebin)

tmp2_ss = output['electron_data8'].copy()
tmp2_ss = tmp2_ss.rebin('pt', pt_rebin)
tmp2_ss = tmp2_ss.rebin('eta', eta_rebin)

tmp3 = scale_and_merge(mc_output['electron_data1'], meta, mc_fileset, nano_mappings, lumi=lumi[year])
tmp3 = tmp3.rebin('pt', pt_rebin)
tmp3 = tmp3.rebin('eta', eta_rebin)

tmp4 = scale_and_merge(mc_output['electron_data2'], meta, mc_fileset, nano_mappings, lumi=lumi[year])
tmp4 = tmp4.rebin('pt', pt_rebin)
tmp4 = tmp4.rebin('eta', eta_rebin)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset', 'phi', 'eta', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('dataset', 'phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset', 'phi', 'eta', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('dataset', 'phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
h3 = Hist1D.from_bincounts(
    tmp1.sum('dataset', 'phi', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('dataset', 'phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
    
h4 = Hist1D.from_bincounts(
    tmp2.sum('dataset', 'phi', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('dataset', 'phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
h5 = Hist1D.from_bincounts(
    tmp1.sum('dataset', 'eta', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('dataset', 'eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
    
h6 = Hist1D.from_bincounts(
    tmp2.sum('dataset', 'eta', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('dataset', 'eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)


h1_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('dataset', 'phi', 'eta', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('dataset', 'phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h2_ss = Hist1D.from_bincounts(
    tmp2_ss.sum('dataset', 'phi', 'eta', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2_ss.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2_ss.sum('dataset', 'phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
h3_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('dataset', 'phi', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('dataset', 'phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
    
h4_ss = Hist1D.from_bincounts(
    tmp2_ss.sum('dataset', 'phi', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2_ss.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp2_ss.sum('dataset', 'phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
h5_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('dataset', 'eta', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('dataset', 'eta', 'pt').values(sumw2=True, overflow = 'all')[()][1].T),
)
    
    
h6_ss = Hist1D.from_bincounts(
    tmp2_ss.sum('dataset', 'eta', 'pt', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2_ss.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp2_ss.sum('dataset', 'eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h1 = h1 * scale_factor
h2 = h2 * scale_factor
h3 = h3 * scale_factor
h4 = h4 * scale_factor
h5 = h5 * scale_factor
h6 = h6 * scale_factor


h1_DY = Hist1D.from_bincounts(
    tmp3.sum('phi', 'eta', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp3.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h2_DY = Hist1D.from_bincounts(
    tmp4.sum('phi', 'eta', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp4.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h3_DY = Hist1D.from_bincounts(
    tmp3.sum('phi', 'pt', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp3.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h4_DY = Hist1D.from_bincounts(
    tmp4.sum('phi', 'pt', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp4.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h5_DY = Hist1D.from_bincounts(
    tmp3.sum('eta', 'pt', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp3.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h6_DY = Hist1D.from_bincounts(
    tmp4.sum('eta', 'pt', overflow = 'all').values(overflow = 'all')[('DY',)].T,
    (tmp4.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)


#h1_DY = h1_DY * scale_factor
#h2_DY = h2_DY * scale_factor
#h3_DY = h3_DY * scale_factor
#h4_DY = h4_DY * scale_factor
#h5_DY = h5_DY * scale_factor
#h6_DY = h6_DY * scale_factor

h1_TT_TW = Hist1D.from_bincounts(
    tmp3.sum('phi', 'eta', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp3.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

h2_TT_TW = Hist1D.from_bincounts(
    tmp4.sum('phi', 'eta', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp4.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

h3_TT_TW = Hist1D.from_bincounts(
    tmp3.sum('phi', 'pt', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp3.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

h4_TT_TW = Hist1D.from_bincounts(
    tmp4.sum('phi', 'pt', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp4.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
    )

h5_TT_TW = Hist1D.from_bincounts(
    tmp3.sum('eta', 'pt', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp3.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

h6_TT_TW = Hist1D.from_bincounts(
    tmp4.sum('eta', 'pt', overflow = 'all').values(overflow = 'all')[('TT/TW',)].T,
    (tmp4.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

    
h1_VV = Hist1D.from_bincounts(
    tmp3.sum('phi', 'eta', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp3.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

h2_VV = Hist1D.from_bincounts(
    tmp4.sum('phi', 'eta', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp4.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('phi', 'eta', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

h3_VV = Hist1D.from_bincounts(
    tmp3.sum('phi', 'pt', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp3.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)


h4_VV = Hist1D.from_bincounts(
    tmp4.sum('phi', 'pt', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp4.axis('eta').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('phi', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

h5_VV = Hist1D.from_bincounts(
    tmp3.sum('eta', 'pt', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp3.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp3.sum('eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)


h6_VV = Hist1D.from_bincounts(
    tmp4.sum('eta', 'pt', overflow = 'all').values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp4.axis('phi').edges(overflow = 'all')),
    errors = np.sqrt(tmp4.sum('eta', 'pt', overflow = 'all').values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

In [ ]:
hists = {
    'pt_data1_OS':  h1,
    'pt_data1_SS':  h1_ss,
    'pt_DY1_OS':    h1_DY,
    'pt_TT_TW1_OS': h1_TT_TW,
    'pt_VV1_OS':    h1_VV,
}

hists['pt_DY1_OS'].label = '$DY\ (OS)$'
hists['pt_DY1_OS'].color = '#FFCA3A'

hists['pt_TT_TW1_OS'].label = '$TT/TW\ (OS)$'
hists['pt_TT_TW1_OS'].color = '#8AC926'

hists['pt_VV1_OS'].label = '$WZ/ZZ\ (OS)$'
hists['pt_VV1_OS'].color = '#C93126'

keys = ['pt_TT_TW1_OS', 'pt_VV1_OS', 'pt_DY1_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_data1_OS']).divide(hists['pt_TT_TW1_OS']+hists['pt_DY1_OS']+hists['pt_VV1_OS'])
ratio = (hists['pt_data1_SS']).divide(hists['pt_data1_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['pt_DY1_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['pt_data1_OS'].counts],
    hists['pt_data1_OS'].edges,
    w2=hists['pt_data1_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['pt_data1_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['pt_data1_SS'].counts],
    hists['pt_data1_SS'].edges,
    w2=hists['pt_data1_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['pt_data1_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2  = ratio.errors**2,
    #w2=[hists['pt_data1_OS'].counts/(hists['pt_TT_TW1_OS'].counts+hists['pt_DY1_OS'].counts+hists['pt_VV1_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$p_{T}\ (leading\ electron)\ (GeV)$')
rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
rax.axhline(y=1, color='r', linestyle='-')

#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zpt(l)_data_ss_'+str(year)+'.png')

In [ ]:
hists = {
    'pt_data2_OS':  h2,
    'pt_data2_SS':  h2_ss,
    'pt_DY2_OS':    h2_DY,
    'pt_TT_TW2_OS': h2_TT_TW,
    'pt_VV2_OS':    h2_VV,
}

hists['pt_DY2_OS'].label = '$DY\ (SS)$'
hists['pt_DY2_OS'].color = '#FFCA3A'

hists['pt_TT_TW2_OS'].label = '$TT/TW\ (SS)$'
hists['pt_TT_TW2_OS'].color = '#8AC926'

hists['pt_VV2_OS'].label = '$WZ/ZZ\ (SS)$'
hists['pt_VV2_OS'].color = '#C93126'

keys = ['pt_TT_TW2_OS', 'pt_VV2_OS', 'pt_DY2_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['pt_data2_OS']).divide(hists['pt_TT_TW2_OS']+hists['pt_DY2_OS']+hists['pt_VV2_OS'])
ratio = (hists['pt_data2_SS']).divide(hists['pt_data2_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['pt_DY2_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['pt_data2_OS'].counts],
    hists['pt_data2_OS'].edges,
    w2=hists['pt_data2_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['pt_data2_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['pt_data2_SS'].counts],
    hists['pt_data2_SS'].edges,
    w2=hists['pt_data2_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['pt_data2_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2=ratio.errors**2,
    #w2=[hists['pt_data2_OS'].counts/(hists['pt_TT_TW2_OS'].counts+hists['pt_DY2_OS'].counts+hists['pt_VV2_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$p_{T}\ (trailing\ electron)\ (GeV)$')
rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)
rax.axhline(y=1, color='r', linestyle='-')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zpt(t)_data_ss_'+str(year)+'.png')

In [ ]:
hists = {
    'eta_data1_OS':  h3,
    'eta_data1_SS':  h3_ss,
    'eta_DY1_OS':    h3_DY,
    'eta_TT_TW1_OS': h3_TT_TW,
    'eta_VV1_OS':    h3_VV,
}

hists['eta_DY1_OS'].label = '$DY\ (SS)$'
hists['eta_DY1_OS'].color = '#FFCA3A'

hists['eta_TT_TW1_OS'].label = '$TT/TW\ (SS)$'
hists['eta_TT_TW1_OS'].color = '#8AC926'

hists['eta_VV1_OS'].label = '$WZ\ (SS)$'
hists['eta_VV1_OS'].color = '#C93126'

keys = ['eta_TT_TW1_OS', 'eta_VV1_OS', 'eta_DY1_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['eta_data1_OS']).divide(hists['eta_TT_TW1_OS']+hists['eta_DY1_OS']+hists['eta_VV1_OS'])
ratio = (hists['eta_data1_SS']).divide(hists['eta_data1_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['eta_DY1_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['eta_data1_OS'].counts],
    hists['eta_data1_OS'].edges,
    w2=hists['eta_data1_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['eta_data1_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['eta_data1_SS'].counts],
    hists['eta_data1_SS'].edges,
    w2=hists['eta_data1_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pbs SS)', np.sum(hists['eta_data1_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['eta_data1_OS'].counts/(hists['eta_TT_TW1_OS'].counts+hists['eta_DY1_OS'].counts+hists['eta_VV1_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$\eta\ (leading\ electron)$')
rax.set_ylabel(r'Obs/Pred')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
#ax.set_yscale('log')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zeta(l)_data_ss_'+str(year)+'.png')

In [ ]:
hists = {
    'eta_data2_OS':  h4,
    'eta_data2_SS':  h4_ss,
    'eta_DY2_OS':    h4_DY,
    'eta_TT_TW2_OS': h4_TT_TW,
    'eta_VV2_OS':    h4_VV,
}

hists['eta_DY2_OS'].label = '$DY\ (SS)$'
hists['eta_DY2_OS'].color = '#FFCA3A'

hists['eta_TT_TW2_OS'].label = '$TT/TW\ (SS)$'
hists['eta_TT_TW2_OS'].color = '#8AC926'

hists['eta_VV2_OS'].label = '$WZ/ZZ\ (SS)$'
hists['eta_VV2_OS'].color = '#C93126'

keys = ['eta_TT_TW2_OS', 'eta_VV2_OS', 'eta_DY2_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['eta_data2_OS']).divide(hists['eta_TT_TW2_OS']+hists['eta_DY2_OS']+hists['eta_VV2_OS'])
ratio = (hists['eta_data2_SS']).divide(hists['eta_data2_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['eta_DY2_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['eta_data2_OS'].counts],
    hists['eta_data2_OS'].edges,
    w2=hists['eta_data2_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['eta_data2_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['eta_data2_SS'].counts],
    hists['eta_data2_SS'].edges,
    w2=hists['eta_data2_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['eta_data2_SS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['eta_data2_OS'].counts/(hists['eta_TT_TW2_OS'].counts+hists['eta_DY2_OS'].counts+hists['eta_VV2_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.7,1.3)
rax.set_xlabel(r'$\eta\ (trailing\ electron)$')
rax.set_ylabel(r'Obs/Pred')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
#ax.set_yscale('log')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zeta(t)_data_ss_'+str(year)+'.png')

In [ ]:
hists = {
    'phi_data1_OS':  h5,
    'phi_data1_SS':  h5_ss,
    'phi_DY1_OS':    h5_DY,
    'phi_TT_TW1_OS': h5_TT_TW,
    'phi_VV1_OS':    h5_VV,
}

hists['phi_DY1_OS'].label = '$DY\ (OS)$'
hists['phi_DY1_OS'].color = '#FFCA3A'

hists['phi_TT_TW1_OS'].label = '$TT/TW\ (OS)$'
hists['phi_TT_TW1_OS'].color = '#8AC926'

hists['phi_VV1_OS'].label = '$WZ/ZZ\ (OS)$'
hists['phi_VV1_OS'].color = '#C93126'

keys = ['phi_TT_TW1_OS', 'phi_VV1_OS', 'phi_DY1_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['phi_data1_OS']).divide(hists['phi_TT_TW1_OS']+hists['phi_DY1_OS']+hists['phi_VV1_OS'])
ratio = (hists['phi_data1_SS']).divide(hists['phi_data1_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['phi_DY1_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['phi_data1_OS'].counts],
    hists['phi_data1_OS'].edges,
    w2=hists['phi_data1_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['phi_data1_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['phi_data1_SS'].counts],
    hists['phi_data1_SS'].edges,
    w2=hists['phi_data1_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['phi_data1_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['phi_data1_OS'].counts/(hists['phi_TT_TW1_OS'].counts+hists['phi_DY1_OS'].counts+hists['phi_VV1_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$\phi\ (leading\ electron)$')
rax.set_ylabel(r'Obs/Pred')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zphi(l)_data_ss_'+str(year)+'.png')

In [ ]:
hists = {
    'phi_data2_OS':  h6,
    'phi_data2_SS':  h6_ss,
    'phi_DY2_OS':    h6_DY,
    'phi_TT_TW2_OS': h6_TT_TW,
    'phi_VV2_OS':    h6_VV,
}

hists['phi_DY2_OS'].label = '$DY\ (SS)$'
hists['phi_DY2_OS'].color = '#FFCA3A'

hists['phi_TT_TW2_OS'].label = '$TT/TW\ (SS)$'
hists['phi_TT_TW2_OS'].color = '#8AC926'

hists['phi_VV2_OS'].label = '$WZ/ZZ\ (SS)$'
hists['phi_VV2_OS'].color = '#C93126'

keys = ['phi_TT_TW2_OS', 'phi_VV2_OS', 'phi_DY2_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['phi_data2_OS']).divide(hists['phi_TT_TW2_OS']+hists['phi_DY2_OS']+hists['phi_VV2_OS'])
ratio = (hists['phi_data2_SS']).divide(hists['phi_data2_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['phi_DY2_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['phi_data2_OS'].counts],
    hists['phi_data2_OS'].edges,
    w2=hists['phi_data2_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['phi_data2_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['phi_data2_SS'].counts],
    hists['phi_data2_SS'].edges,
    w2=hists['phi_data2_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['phi_data2_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['phi_data2_OS'].counts/(hists['phi_TT_TW2_OS'].counts+hists['phi_DY2_OS'].counts+hists['phi_VV2_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$\phi\ (trailing\ electron)$')
rax.set_ylabel(r'Obs/Pred')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
ax.set_yscale('log')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/zphi(t)_data_ss_'+str(year)+'.png')

In [ ]:
tmp1 = output['MET'].copy()
tmp1 = tmp1.rebin('pt', met_pt)

tmp1_ss = output['MET4'].copy()
tmp1_ss = tmp1_ss.rebin('pt', met_pt)

tmp2 = scale_and_merge(mc_output['MET'], samples, mc_fileset, nano_mappings, lumi=lumi[year])
tmp2 = tmp2.rebin('pt', met_pt)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset', overflow='all').values(overflow='all')[()].T,
    (tmp1.axis('pt').edges(overflow='all')),
    errors = np.sqrt(tmp1.sum('dataset', overflow='all').values(sumw2=True, overflow='all')[()][1].T),
)

h1 = h1 * scale_factor

h1_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('dataset', overflow='all').values(overflow='all')[()].T,
    (tmp1_ss.axis('pt').edges(overflow='all')),
    errors = np.sqrt(tmp1_ss.sum('dataset', overflow='all').values(sumw2=True, overflow='all')[()][1].T),
)

h1_DY = Hist1D.from_bincounts(
    tmp2.values(overflow='all')[('DY',)].T,
    (tmp2.axis('pt').edges(overflow='all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow='all')[('DY',)][1].T),
)


h1_TT_TW = Hist1D.from_bincounts(
    tmp2.values(overflow='all')[('TT/TW',)].T,
    (tmp2.axis('pt').edges(overflow='all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow='all')[('TT/TW',)][1].T),
)

h1_VV = Hist1D.from_bincounts(
    tmp2.values(overflow='all')[('WZ/ZZ',)].T,
    (tmp2.axis('pt').edges(overflow='all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow='all')[('WZ/ZZ',)][1].T),
)

In [ ]:
hists = {'MET_pt_data_OS': h1,
         'MET_pt_data_SS': h1_ss,
         'MET_pt_DY_OS': h1_DY,
         'MET_pt_TT_TW_OS': h1_TT_TW,
         'MET_pt_VV_OS': h1_VV,
}

hists['MET_pt_DY_OS'].label = '$DY\ (SS)$'
hists['MET_pt_DY_OS'].color = '#FFCA3A'

hists['MET_pt_TT_TW_OS'].label = '$TT/TW\ (SS)$'
hists['MET_pt_TT_TW_OS'].color = '#8AC926'

hists['MET_pt_VV_OS'].label = '$WZ/ZZ\ (SS)$'
hists['MET_pt_VV_OS'].color = '#C93126'

keys = ['MET_pt_TT_TW_OS', 'MET_pt_VV_OS', 'MET_pt_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['MET_pt_data_OS']).divide(hists['MET_pt_TT_TW_OS']+hists['MET_pt_DY_OS']+hists['MET_pt_VV_OS'])
ratio = (hists['MET_pt_data_SS']).divide(hists['MET_pt_data_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['MET_pt_DY_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['MET_pt_data_OS'].counts],
    hists['MET_pt_data_OS'].edges,
    w2=hists['MET_pt_data_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['MET_pt_data_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['MET_pt_data_SS'].counts],
    hists['MET_pt_data_SS'].edges,
    w2=hists['MET_pt_data_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['MET_pt_data_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['MET_pt_data_OS'].counts/(hists['MET_pt_TT_TW_OS'].counts+hists['MET_pt_DY_OS'].counts+hists['MET_pt_VV_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$p_{T}\ (MET)$')
rax.set_ylabel(r'Obs/Pred')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
ax.set_yscale('log')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/metpt_data_ss_'+str(year)+'.png')

In [ ]:
tmp1 = output['N_jet'].copy()
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

tmp1_ss = output['N_jet4'].copy()
tmp1_ss = tmp1_ss.rebin('multiplicity', N_bins_red)

tmp2 = scale_and_merge(mc_output['N_jet'], meta, mc_fileset, nano_mappings, lumi=lumi[year])
tmp2 = tmp2.rebin('multiplicity', N_bins_red)

h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h1 = h1 * scale_factor

h1_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h1_DY = Hist1D.from_bincounts(
    tmp2.values(overflow = 'all')[('DY',)].T,
    (tmp2.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h1_TT_TW = Hist1D.from_bincounts(
    tmp2.values(overflow = 'all')[('TT/TW',)].T,
    (tmp2.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)

    
h1_VV = Hist1D.from_bincounts(
    tmp2.values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp2.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

In [ ]:
hists = {'N_jet_data_OS'  : h1,
         'N_jet_data_SS'  : h1_ss,
         'N_jet_DY_OS'    : h1_DY,
         'N_jet_TT_TW_OS' : h1_TT_TW,
         'N_jet_VV_OS'    : h1_VV,
}

hists['N_jet_DY_OS'].label = '$DY\ (SS)$'
hists['N_jet_DY_OS'].color = '#FFCA3A'

hists['N_jet_TT_TW_OS'].label = '$TT/TW\ (SS)$'
hists['N_jet_TT_TW_OS'].color = '#8AC926'

hists['N_jet_VV_OS'].label = '$WZ/ZZ\ (SS)$'
hists['N_jet_VV_OS'].color = '#C93126'

keys = ['N_jet_TT_TW_OS', 'N_jet_VV_OS', 'N_jet_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['N_jet_data_OS']).divide(hists['N_jet_TT_TW_OS']+hists['N_jet_DY_OS']+hists['N_jet_VV_OS'])
ratio = (hists['N_jet_data_SS']).divide(hists['N_jet_data_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['N_jet_DY_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['N_jet_data_OS'].counts],
    hists['N_jet_data_OS'].edges,
    w2=hists['N_jet_data_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['N_jet_data_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['N_jet_data_SS'].counts],
    hists['N_jet_data_SS'].edges,
    w2=hists['N_jet_data_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['N_jet_data_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2, 
    #w2=[hists['N_jet_data_OS'].counts/(hists['N_jet_TT_TW_OS'].counts+hists['N_jet_DY_OS'].counts+hists['N_jet_VV_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$N_{jets}$')
rax.set_ylabel(r'Data/MC')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/njet_data_ss_'+str(year)+'.png')

In [ ]:
tmp1 = output['PV_npvsGood'].copy()
tmp1_ss = output['PV_npvsGood4'].copy()
tmp2 = scale_and_merge(mc_output['PV_npvsGood'], meta, mc_fileset, nano_mappings, lumi=lumi[year])

h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h1 = h1 * scale_factor

h1_ss = Hist1D.from_bincounts(
    tmp1_ss.sum('dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1_ss.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1_ss.sum('dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)

h1_DY = Hist1D.from_bincounts(
    tmp2.values(overflow = 'all')[('DY',)].T,
    (tmp2.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('DY',)][1].T),
)

h1_TT_TW = Hist1D.from_bincounts(
    tmp2.values(overflow = 'all')[('TT/TW',)].T,
    (tmp2.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('TT/TW',)][1].T),
)


h1_VV = Hist1D.from_bincounts(
    tmp2.values(overflow = 'all')[('WZ/ZZ',)].T,
    (tmp2.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('WZ/ZZ',)][1].T),
)

In [ ]:
hists = {'PV_nGood_data_OS': h1,
         'PV_nGood_data_SS': h1_ss,
         'PV_nGood_DY_OS': h1_DY,
         'PV_nGood_TT_TW_OS': h1_TT_TW,
         'PV_nGood_VV_OS': h1_VV
}

hists['PV_nGood_DY_OS'].label = '$DY\ (SS)$'
hists['PV_nGood_DY_OS'].color = '#FFCA3A'

hists['PV_nGood_TT_TW_OS'].label = '$TT/TW\ (SS)$'
hists['PV_nGood_TT_TW_OS'].color = '#8AC926'

hists['PV_nGood_VV_OS'].label = '$WZ/ZZ\ (SS)$'
hists['PV_nGood_VV_OS'].color = '#C93126'

keys = ['PV_nGood_TT_TW_OS', 'PV_nGood_VV_OS', 'PV_nGood_DY_OS']

total_mc = get_total(hists, keys)

#ratio = (hists['PV_nGood_data_OS']).divide(hists['PV_nGood_TT_TW_OS']+hists['PV_nGood_DY_OS']+hists['PV_nGood_VV_OS'])
ratio = (hists['PV_nGood_data_SS']).divide(hists['PV_nGood_data_OS'])

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    'Preliminary',
    data=True,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys ],
    hists['PV_nGood_DY_OS'].edges,
    w2=[hists[x].errors**2 for x in keys ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
    color=[ hists[x].color for x in keys ],
    ax=ax)

hep.histplot(
    [hists['PV_nGood_data_OS'].counts],
    hists['PV_nGood_data_OS'].edges,
    w2=hists['PV_nGood_data_OS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (pred SS)', np.sum(hists['PV_nGood_data_OS'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['PV_nGood_data_SS'].counts],
    hists['PV_nGood_data_SS'].edges,
    w2=hists['PV_nGood_data_SS'].errors**2,
    histtype="errorbar",
    stack=False,
    label=['%s (%.0f)'%('Data (obs SS)', np.sum(hists['PV_nGood_data_SS'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [ratio.counts],
    ratio.edges,
    w2 = ratio.errors**2,
    #w2=[hists['PV_nGood_data_OS'].counts/(hists['PV_nGood_TT_TW_OS'].counts+hists['PV_nGood_DY_OS'].counts+hists['PV_nGood_VV_OS'].counts)**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.5)
rax.set_xlabel(r'$PV\ N_{good}$')
rax.set_ylabel(r'Obs/Pred')
rax.axhline(y=1, color='r', linestyle='-')

ax.set_ylabel(r'Events')
ax.set_yscale('log')

#add_uncertainty(total_mc, rax, ratio=True)

ax.legend(prop={"size":15})

plt.show()

fig.savefig('/home/users/ewallace/public_html/FCNC/pvngood_data_ss_'+str(year)+'.png')